In [1]:
import numpy as np
import csv

import nbimporter

import s04_07_b_gas_boiler as _04_07_AppB_gas
import s04_07_a_kerosene_boiler as _04_07_AppA_ker
import s07_c_gas_boiler as _07_AppC_gas
import s07_d_kerosene_boiler as _07_AppD_ker

Importing Jupyter notebook from s04_07_b_gas_boiler.ipynb
Importing Jupyter notebook from s04_07_a_kerosene_boiler.ipynb
Importing Jupyter notebook from s07_c_gas_boiler.ipynb
Importing Jupyter notebook from s07_d_kerosene_boiler.ipynb


# Chapter VIII Cogeneration System / コージェネレーション設備

## 1. Introduction

This calculation depends on the specification of "Chapter VIII Cogeneration System" in BRI

## 2. Cogeneration Spec

In [2]:
def convert_specdata_to_spec(pu_id, tu_id, bb_id):

    def read_registered_data(file_name):
        
        with open(file_name, 'r', encoding = 'utf8') as f:
            reader = csv.reader(f)
            header = next(reader)
            return [row for row in reader]
    
    def search_matched_data(speclist, pu_id, tu_id, bb_id):
        
        selected = [r for r in speclist if (pu_id == r[3] and tu_id == r[5] and bb_id == r[7])]
        if not len(selected) == 1:
            raise Exception('error; More than one matched data are found.')
        return selected[0]
    
    def make_dictionary(d):
        return {
            'PU_GenerationMethod' : { '電主' : 'ElectricityPriority', '熱主' : 'HeatPriority' }[d[13]],
            'PU_virtualGeneration' : {
                'a_PU' : d[14], 'a_DHW' : d[15], 'a_HWH' : d[16], 'b' : d[17], 'c' : d[18]
            },
            'PU_virtualFuel' : {
                'a_DHW' : d[19], 'a_HWH' : d[20]
            },
            'PU_UpperCoef' : {
                'a_DHW' : d[21], 'a_HWH' : d[22], 'b' : d[23]
            },
            'PU_GenIta' : {
                'a_PU' : d[24], 'a_DHW' : d[25], 'a_HWH' : d[26], 'b' : d[27], 'UpperLimit' : d[28], 'LowerLimit' : d[29]
            },
            'PU_ExaustIta' : {
                'a_PU' : d[30], 'a_DHW' : d[31], 'a_HWH' : d[32], 'b' : d[33], 'UpperLimit' : d[34], 'LowerLimit' : d[35]
            },
            'BB_HW' : {
                'Type' : { 'ガス' : 'gas', '灯油' : 'kerosene' }[d[36]],
                'RatedEfficiency' : d[37]
            },
            'BB_WH' : {
                'Type' : { 'G_EJ' : 'gasConv' }[d[38]],
                'RatedEfficiency' : d[39],
                'RatedPower' : d[40]
            },
            'ExaustUseForWaterHeating' : { 'あり' : 'Yes', 'なし' : 'No' }[d[41]],
            'ExaustUsePriority' : { '暖房優先' : 'WaterHeating', '給湯優先' : 'HotWater', 'ND' : 'ND' }[d[42]],
            'PU_exaust_rate' : {
                'HotWater' : d[43], 'WaterHeating' : d[44]
            },
            'TU_auxPower' :{
                'WaterHeating' : d[45],
                'HotWater' : d[46]
            },
            'RatedPower' : d[47]
        }

    FILE_NAME = 'data/cog_data.csv'
    
    speclist = read_registered_data(FILE_NAME)
    selected = search_matched_data(speclist, pu_id, tu_id, bb_id)
    return make_dictionary(selected)
    

spec = convert_specdata_to_spec(pu_id='FCCS07B2N', tu_id='-', bb_id='FT4216ARS0AW3P')
print(spec)

{'PU_GenerationMethod': 'ElectricityPriority', 'PU_virtualGeneration': {'a_PU': '1.1462', 'a_DHW': '0', 'a_HWH': '0', 'b': '-7.6278', 'c': '0.9705'}, 'PU_virtualFuel': {'a_DHW': 'ND', 'a_HWH': 'ND'}, 'PU_UpperCoef': {'a_DHW': 'ND', 'a_HWH': 'ND', 'b': 'ND'}, 'PU_GenIta': {'a_PU': '0.003684', 'a_DHW': '0', 'a_HWH': '0', 'b': '0.2788', 'UpperLimit': '0.4389', 'LowerLimit': '0.4192'}, 'PU_ExaustIta': {'a_PU': '-0.000617', 'a_DHW': '0', 'a_HWH': '0', 'b': '0.362', 'UpperLimit': '0.3429', 'LowerLimit': '0.3278'}, 'BB_HW': {'Type': 'gas', 'RatedEfficiency': '0.905'}, 'BB_WH': {'Type': 'gasConv', 'RatedEfficiency': '0.87', 'RatedPower': '17400'}, 'ExaustUseForWaterHeating': 'No', 'ExaustUsePriority': 'ND', 'PU_exaust_rate': {'HotWater': '0.4432', 'WaterHeating': 'ND'}, 'TU_auxPower': {'WaterHeating': 'ND', 'HotWater': '7'}, 'RatedPower': '700'}


In [3]:
def get_default_spec(cgs_name):
    return {
        'GEC1'  : {
            'ExaustUseForWaterHeating' : 'Yes',
            'ExaustUsePriority' : 'HotWater',
            'BB_HW' : {'Type' : 'gas', 'RatedEfficiency' : 0.782},
            'BB_WH' : {'Type' : 'gasConv'      , 'RatedEfficiency' : 0.82, 'RatedPower' : 17400},
            'PU_exaust_rate' : {'HotWater' : 0.7494, 'WaterHeating' : 0.7758},
            'PU_GenerationMethod' : 'HeatPriority'            ,
            'PU_virtualGeneration' : {'a_PU' : 0.0000, 'a_DHW' : 0.1398, 'a_HWH' : 0.1398, 'b' :   7.7827, 'c' : 0.6925},
            'PU_virtualFuel' : {'a_DHW' : 1, 'a_HWH' : 0},
            'PU_UpperCoef' : {'a_DHW' : -0.0075, 'a_HWH' : -0.0075, 'b' : 1.4847},
            'PU_GenIta' : {'a_PU' : 0.000000, 'a_DHW' : -0.000034, 'a_HWH' : -0.000034, 'b' : 0.1779, 'UpperLimit' : 0.2069, 'LowerLimit' : 0.1499},
            'PU_ExaustIta' : {'a_PU' : 0.000000, 'a_DHW' : 0.000342, 'a_HWH' : 0.000342, 'b' : 0.5787, 'UpperLimit' : 0.5787, 'LowerLimit' : 0.5576},
            'RatedPower' : 1000,
            'TU_auxPower' :{'HotWater' : 15.1, 'WaterHeating' : 132.1}
        },
        'PEFC5' : {
            'ExaustUseForWaterHeating' : 'Yes',
            'ExaustUsePriority' : 'HotWater',
            'BB_HW' : {'Type' : 'gas', 'RatedEfficiency' : 0.782},
            'BB_WH' : {'Type' : 'gasConv'      , 'RatedEfficiency' : 0.82, 'RatedPower' : 17400},
            'PU_exaust_rate' : {'HotWater' : 0.9654, 'WaterHeating' : 0.0000},
            'PU_GenerationMethod' : 'HeatPriority'            ,
            'PU_virtualGeneration' : {'a_PU' : 1.2469, 'a_DHW' : 0.0000, 'a_HWH' : 0.0000, 'b' : -14.9213, 'c' : 0.9950},
            'PU_virtualFuel' : {'a_DHW' : 1, 'a_HWH' : 0},
            'PU_UpperCoef' : {'a_DHW' :  0.0000, 'a_HWH' :  0.0000, 'b' : 1.0526},
            'PU_GenIta' : {'a_PU' : 0.003096, 'a_DHW' :  0.000000, 'a_HWH' :  0.000000, 'b' : 0.1483, 'UpperLimit' : 0.3193, 'LowerLimit' : 0.2301},
            'PU_ExaustIta' : {'a_PU' : 0.000000, 'a_DHW' : 0.000452, 'a_HWH' : 0.000000, 'b' : 0.3017, 'UpperLimit' : 0.3716, 'LowerLimit' : 0.2803},
            'RatedPower' : 700 ,
            'TU_auxPower' :{'HotWater' : 15.4, 'WaterHeating' : 138.1}
        },
        'SOFC1' : {
            'ExaustUseForWaterHeating' : 'No',
            'ExaustUsePriority' : 'N/A'      ,
            'BB_HW' : {'Type' : 'gas', 'RatedEfficiency' : 0.905},
            'BB_WH' : {'Type' : 'gasCondensing', 'RatedEfficiency' : 0.87, 'RatedPower' : 17400},
            'PU_exaust_rate' : {'HotWater' : 0.7227, 'WaterHeating' : 0.0000},
            'PU_GenerationMethod' : 'ElectricityPriority'     ,
            'PU_virtualGeneration' : {'a_PU' : 1.1175, 'a_DHW' : 0.0000, 'a_HWH' : 0.0000, 'b' :  -6.6385, 'c' : 0.9950},
            'PU_virtualFuel' : {'a_DHW' : 0, 'a_HWH' : 0},
            'PU_UpperCoef' : {'a_DHW' :  0     , 'a_HWH' :  0     , 'b' : 0     },
            'PU_GenIta' : {'a_PU' : 0.003600, 'a_DHW' :  0.000000, 'a_HWH' :  0.000000, 'b' : 0.1951, 'UpperLimit' : 0.3925, 'LowerLimit' : 0.3092},
            'PU_ExaustIta' : {'a_PU' : 0.000500, 'a_DHW' : 0.000000, 'a_HWH' : 0.000000, 'b' : 0.3135, 'UpperLimit' : 0.3539, 'LowerLimit' : 0.3169},
            'RatedPower' : 700 ,
            'TU_auxPower' :{'HotWater' : 17.1, 'WaterHeating' : 0    }
        },
        'SOFC2' : {
            'ExaustUseForWaterHeating' : 'No' ,
            'ExaustUsePriority' : 'N/A'     ,
            'BB_HW' : {'Type' : 'gas', 'RatedEfficiency' : 0.905},
            'BB_WH' : {'Type' : 'gasCondensing', 'RatedEfficiency' : 0.87, 'RatedPower' : 17400},
            'PU_exaust_rate' : {'HotWater' : 0.6885, 'WaterHeating' : 0.0000},
            'PU_GenerationMethod' : 'ElectricityPriority'     ,
            'PU_virtualGeneration' : {'a_PU' : 1.1262, 'a_DHW' : 0.0000, 'a_HWH' : 0.0000, 'b' :  -6.5572, 'c' : 0.9846},
            'PU_virtualFuel' : {'a_DHW' : 0, 'a_HWH' : 0},
            'PU_UpperCoef' : {'a_DHW' :  0     , 'a_HWH' :  0     , 'b' : 0     },
            'PU_GenIta' : {'a_PU' : 0.005800, 'a_DHW' :  0.000000, 'a_HWH' :  0.000000, 'b' : 0.1611, 'UpperLimit' : 0.4290, 'LowerLimit' : 0.3503},
            'PU_ExaustIta' : {'a_PU' : 0.002800, 'a_DHW' : 0.000000, 'a_HWH' : 0.000000, 'b' : 0.1854, 'UpperLimit' : 0.3179, 'LowerLimit' : 0.2756},
            'RatedPower' : 700 ,
            'TU_auxPower' :{'HotWater' : 11.8, 'WaterHeating' : 0    }
        }
    }[cgs_name]

In [4]:
def get_spec(cgs):
    if cgs['Identified'] == 'Default':
        return get_default_spec(cgs['DefaultName'])
    elif cgs['Identified'] == 'Registered':
        return convert_specdata_to_spec(cgs['ID']['PowerUnit'], cgs['ID']['TankUnit'], cgs['ID']['BackupBoiler'])

## 5. Gas Consumption and Kerosene Consumption / ガス消費量及び灯油消費量

$$
\displaystyle
E_{CG} = E_{F,PU} + E_{F,BB,DHW} + E_{F,BB,HWH}
$$

Where  
$E_{CG}$ is the daily primary energy consumption of cogeneration system / 1日当たりのコージェネレーション設備の一次エネルギー消費量, MJ/d;  
$E_{F,PU}$ is the daily fuel consumption of power generation unit / 1日当たりの発電ユニットの燃料消費量, MJ/d;  
$E_{F,BB,DHW}$ is the daily fuel consumption of backup boiler during hot water supply / 1日当たりの給湯時のバックアップボイラーの燃料消費量, MJ/d;  
$E_{F,BB,HWH}$ is the daily fuel consumption of backup boiler during hot water heating / 1日当たりの温水暖房時のバックアップボイラーの燃料消費量, MJ/d.  

In [5]:
# gas consumption and kerosene consumption, MJ/d (eq.1)
def get_energy_cons(fuel_cons_pu, fuel_cons_bb_dhw, fuel_cons_hwh):
    return fuel_cons_pu + fuel_cons_bb_dhw + fuel_cons_hwh

## 6. Amount of Power Generation / 発電量

$$
\displaystyle
E_{E,CG,gen,t} = E_{E,gen,PU,t} - E_{E,TU,aux,t}
$$


Where  
$E_{E,CG,gen,t}$ is the hourly amount of power generation by cogeneration system / 1時間当たりのコージェネレーション設備による発電量, kWh/h;  
$E_{E,gen,PU,t}$ is the hourly amount of power generation unit / 1時間当たりの発電ユニットの発電量, kWh/h;  
$E_{E,TU,aux,t}$ is the hourly auxiliary machine power consumption of the tank unit / 1時間当たりのタンクユニットの補機消費電力量, kWh/h.  

In [6]:
# amount of power generation, kWh/h (eq.2)
def get_pow_gen(pow_gen_pu, tu_aux_pc):
    return [pow_gen_pu[t] - tu_aux_pc[t] for t in range(24)]

## 7.Annual Energy Efficiency of Buckup Boiler of Hot Water Supply / 給湯時のバックアップボイラーの年間平均効率

$$
\displaystyle
e_{BB,ave} = \frac {\sum^{365}_{d=1} L_{BB,DHW}} {\sum^{365}_{d=1} E_{F,BB,DHW}}
$$
where  
$L_{BB,DHW}$ is the daily hot water supply heat load shared by backup boiler / 1日当たりの給湯のバックアップボイラーが分担する給湯熱負荷, MJ/d;  
$E_{BB,DHW}$ is the daily fuel consumption by backup boiler / 1日あたりの給湯のバックアップボイラーの燃料消費量, MJ/d.

In [7]:
# eq.3
def get_e_BB_ave(load_bb_dhw, fuel_cons_bb_dhw):
    return sum(load_bb_dhw) / sum(fuel_cons_bb_dhw)

#### Example

In [8]:
_L_BB_DHW = [5]*365
_E_F_BB_DHW = [10]*365
get_e_BB_ave(_L_BB_DHW, _E_F_BB_DHW)

0.5

## 8. Fuel Consumption of Backup Boiler of Hot Water Supply / 給湯時のバックアップボイラーの燃料消費量

### 8.1 Fuel Consumption / 燃料消費量

This calculation depends on the methods for the gas boiler and the kerosene boiler for hot water supply.

### 8.2 Hot Water Heat Load Shared by Backup Boiler / バックアップボイラーが分担する給湯熱負荷

$$
\displaystyle
L_{BB,DHW,k} = L_{BB,DHW} \times \frac{L''_{k}}{L_{DHW}}
$$

$$
\displaystyle
L_{BB,DHW,s} = L_{BB,DHW} \times \frac{L''_{s}}{L_{DHW}}
$$

$$
\displaystyle
L_{BB,DHW,w} = L_{BB,DHW} \times \frac{L''_{w}}{L_{DHW}}
$$

$$
\displaystyle
L_{BB,DHW,b1} = L_{BB,DHW} \times \frac{L''_{b1}}{L_{DHW}}
$$

$$
\displaystyle
L_{BB,DHW,b2} = L_{BB,DHW} \times \frac{L''_{b2}}{L_{DHW}}
$$

$$
\displaystyle
L_{BB,DHW,ba1} = L_{BB,DHW} \times \frac{L''_{ba1}}{L_{DHW}}
$$

$$
\displaystyle
L_{BB,DHW,ba2} = L''_{ba2}
$$


Where  
$L_{BB,DHW}$ is the daily hot water supply heat load shared by backup boiler / 1日当たりの給湯のバックアップボイラーが分担する給湯熱負荷, MJ/d;  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷, MJ/d;  
$L''_{k}$ is the daily hot water supply hea load corrected for solar heat by the kitchen / 1日当たりの台所水栓における太陽熱補正給湯熱負荷, MJ/d;  
$L''_{w}$ is the daily hot water supply hea load corrected for solar heat by the washroom / 1日当たりの洗面水栓における太陽熱補正給湯熱負荷, MJ/d;  
$L''_{s}$ is the daily hot water supply hea load corrected for solar heat by the shower / 1日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷, MJ/d;  
$L''_{b1}$ is the daily hot water supply hea load corrected for solar heat by filling the bathtub manually / 1日当たりの浴槽水栓湯はりにおける太陽熱補正給湯熱負荷, MJ/d;  
$L''_{b2}$ is the daily hot water supply hea load corrected for solar heat by filling the bathtub automattically / 1日当たりの浴槽自動湯はりにおける太陽熱補正給湯熱負荷, MJ/d;  
$L''_{ba1}$ is the daily hot water supply hea load corrected for solar heat by addition of hot water to bathtub / 1日当たりの浴槽水栓さし湯における太陽熱補正給湯熱負荷, MJ/d.

$$
\displaystyle
L_{BB,DHW} = L_{DHW} - Q_{gen,DHW}
$$

Where  
$Q_{gen,DHW}$ is the daily exhaust heat utilization of hot water supply / 1日当たりの給湯の排熱利用量 MJ/d.  

In [9]:
def get_bb_fuel_cons(spec, temp_ex_ave, l2d_k, l2d_s, l2d_w, l2d_b1, l2d_b2, l2d_ba1, l2d_ba2, l_hws, Q_gen_DHW):
    
    bb_hws_spec = spec['BB_HW']
    
    # hot water heat load shared by backup boiler, MJ/d (eq.5)
    l_bb_hws = l_hws - Q_gen_DHW
    
    # hot water heat load shared by backup boiler of each usage, MJ/d (eq.4)
    l_bb_hws_k   = l_bb_hws * l2d_k   / l_hws
    l_bb_hws_s   = l_bb_hws * l2d_s   / l_hws
    l_bb_hws_w   = l_bb_hws * l2d_w   / l_hws
    l_bb_hws_b1  = l_bb_hws * l2d_b1  / l_hws
    l_bb_hws_b2  = l_bb_hws * l2d_b2  / l_hws
    l_bb_hws_ba1 = l_bb_hws * l2d_ba1 / l_hws
    l_bb_hws_ba2 = l2d_ba2    
    
    # fuel consumption, MJ/d
    if bb_hws_spec['Type'] == 'gas':
        return _07_AppC_gas.get_gas_cons(
            hws_type='AutoReheat', e_rtd=bb_hws_spec['RatedEfficiency'], temp_ex_ave=temp_ex_ave,
            l2d_k=l_bb_hws_k, l2d_s=l_bb_hws_s, l2d_w=l_bb_hws_w, l2d_b1=l_bb_hws_b1, l2d_b2=l_bb_hws_b2, l2d_ba1=l_bb_hws_ba1, l2d_ba2=l_bb_hws_ba2
        )
    elif bb_hws_spec['Type'] == 'kerosene':
        return _07_AppD_ker.get_kerosene_cons(
            hws_type='AutoReheat', e_rtd=bb_hws_spec['RatedEfficiency'], temp_ex_ave=temp_ex_ave,
            l2d_k=l_bb_hws_k, l2d_s=l_bb_hws_s, l2d_w=l_bb_hws_w, l2d_b1=l_bb_hws_b1, l2d_b2=l_bb_hws_b2, l2d_ba1=l_bb_hws_ba1, l2d_ba2=l_bb_hws_ba2
        )
    else:
        raise ValueError('バックアップボイラーの燃料種別に指定された値が不正です。')

## 9.Fuel Consumption of Backup Boiler Hot Water Heating Supply / 温水暖房時のバックアップボイラーの燃料消費量

$$
\displaystyle
E_{F,BB,HWH} = \sum_t E_{F,BB,HWH,t}
$$

Where  
$E_{F,BB,HWH}$ is the daily fuel consumption of backup boiler during hot water heating / 1日当たりの温水暖房時のバックアップボイラーの燃料消費量, MJ/d;  
$E_{F,BB,HWH,t}$ is the hourly fuel consumption of backup boiler during hot water heating at $t$ o'clock / 時刻$t$における1時間当たりの温水暖房時のバックアップボイラーの燃料消費量, MJ/h.  

$$
\displaystyle
Q_{out,BB,HWH,t} = \min \left( L_{BB,HWH,t}, Q_{max,BB,HWH} \right)
$$

$$
\displaystyle
L_{BB,HWH,t} = \left\{ \begin{array}{ll}
    L_{HWH,t} - Q_{gen,HWH} \times L_{HWH,t} \div L_{HWH} & (L_{HWH} \neq 0)\\ 
    0 & (L_{HWH} = 0)\\ 
    \end{array}
\right.
$$

Where  
$L_{BB,HWH,t}$ is the hourly heat load of hot water heating shared by backup boiler during hot water heating / 1時間当たりの温水暖房時のバックアップボイラーが分担する温水暖房の熱負荷, MJ/h;  
$Q_{max,BB,HWH}$ is the hourly maximum output of backup boiler during hot water heating / 1時間当たりの温水暖房時のバックアップボイラーの最大出力, MJ/h;  
$L_{HWH,t}$ is the hourly heat load of hot water heating at $t$ o'clock / 時刻$t$における1時間当たりの温水暖房の熱負荷, MJ/h;    
$Q_{gen,HWH}$ is the daily exhaust heat consumption of hot water heating / 1日当たりの温水暖房の排熱利用量, MJ/d;   
$L_{HWH}$ is the daily heat load of hot water heating / 1日当たりの温水暖房の熱負荷, MJ/d.    

In [10]:
# 式(7)
def get_q_out_bb_hwh(spec, l_hwh, q_gen_hwh):

    hs_type = spec['BB_WH']['Type']
    p_rtd = spec['BB_WH']['RatedPower']
    
    # daily hot water heating demand
    l_hwh_d = get_L_HWH(l_hwh)

    # hourly heat load of hot water heating shared by backup boiler, MJ/d (eq.8)
    l_bb_hwh = [h - q_gen_hwh * h / l_hwh_d for h in l_hwh] if l_hwh_d > 0.0 else [0.0]*24

    # function for calculation of maximum output, MJ/h
    f_q_max = {
        'kerosene' : _04_07_AppA_ker.get_Q_max_H_hs,
        'gasCondensing' : _04_07_AppB_gas.get_Q_max_H_hs,
        'gasConv' : _04_07_AppB_gas.get_Q_max_H_hs
    }[hs_type]
    
    return [min(h, f_q_max(p_rtd)) for h in l_bb_hwh]

In [11]:
def get_bb_hwh_fuel_cons(l_hwh, q_gen_hwh, spec, temp_sw):
    
    hs_type = spec['BB_WH']['Type']
    eff_rtd = spec['BB_WH']['RatedEfficiency']
    p_rtd = spec['BB_WH']['RatedPower']

    q_out_bb_hwh = get_q_out_bb_hwh(spec, l_hwh, q_gen_hwh)
    
    temp_rw = 40.0 # 暫定
    
    f_fuel_cons = {
        'kerosene' : _04_07_AppA_ker.get_kerosene_cons,
        'gasCondensing' : _04_07_AppB_gas.get_gas_cons,
        'gasConv' : _04_07_AppB_gas.get_gas_cons
    }[hs_type]
    
    bb_hwh_fuel_cons = [f_fuel_cons(hs_type, q, t, temp_rw, eff_rtd, p_rtd) for (q,t) in zip(q_out_bb_hwh, temp_sw)]
    
    return sum(bb_hwh_fuel_cons)

## 10.Power Generation Unit / 発電ユニット

### 10.1 Exhaust Heat Utilization / 排熱利用量

The daily exhaust heat utilization $Q_{gen,DHW}$ of hot water supply and the daily exhaust heat utilization $Q_{gen,HWH}$ of hot water heating are calculated by the equations below, depending on whether exhaust heat is used for hot water heating and how exhaust heat is used.   
Whether exhaust heat is used for hot water heating and how exhaust heat is used are in accordance with Appendix A.

i) When there is no use of exhaust heat to hot water heating:

$$
\displaystyle
Q_{gen,DHW} = \min \left( Q_{PU,gen} \times r_{DHW,gen,PU},L_{DHW} \right)
$$

$$
\displaystyle
Q_{gen,HWH} = 0.0
$$

ii) In case of using exhaust heat to hot water heating, models with hot water supply priority:

$$
\displaystyle
Q_{gen,DHW} = \min \left( Q_{PU,gen} \times r_{DHW,gen,PU},L_{DHW} \right)
$$

$$
\displaystyle
Q_{gen,HWH} = \min \left( (Q_{PU,gen} - Q_{gen,DHW}) \times r_{HWH,gen,PU},L_{HWH} \right)
$$

iii) In case of using exhaust heat to hot water heating, models with hot water heating priority: <br><br>

$$
\displaystyle
Q_{gen,HWH} = \min \left( Q_{PU,gen} \times r_{HWH,gen,PU},L_{HWH} \right)
$$

$$
\displaystyle
Q_{gen,DHW} = \min \left( (Q_{PU,gen} - Q_{gen,HWH}) \times r_{DHW,gen,PU},L_{DHW} \right)
$$

Where  
$Q_{PU,gen}$ is the daily exhaust heat quantity of power generation unit. / 1日当たりの発電ユニット排熱量(MJ/day)  
$r_{DHW,gen,PU}$ is the exhaust heat utilization rate at hot water supply in power generation unit. / 発電ユニットの給湯排熱利用率  
$r_{HWH,gen,PU}$ is the exhaust heat utilization rate at hot water heating in power generation unit. / 発電ユニットの温水暖房排熱利用率  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  

The exhaust heat utilization rate $r_{DHW,gen,PU}$ at hot water supply and the exhaust heat utilization rate $r_{HWH,gen,PU}$ at hot water heating in power generation unit are in accordance with Appendix A.

$$
\displaystyle
Q_{PU,gen} = E_{F,PU} \times e_{H,PU}
$$

Where  
$E_{F,PU}$ is the daily fuel consumption of power generation unit / 1日当たりの発電ユニットの燃料消費量, MJ/d;  
$e_{H,PU}$ is the daily average exhaust heat efficiency of power generation unit / 発電ユニットの日平均排熱効率.  

In [12]:
# 式(9)-(11)
def get_Q_gen_DHWandHWH(fuel_cons_pu, exhst_h_eff, spec, l_dhw, l_hwh):
    
    # daily hot water heating demand
    l_hwh_d = get_L_HWH(l_hwh)

    # daily exhaust heat quantity of power generation unit, MJ/d (eq.12)
    q_pu_gen = fuel_cons_pu * exhst_h_eff
    
    exaust_use_WH = spec['ExaustUseForWaterHeating']
    exaust_priority = spec['ExaustUsePriority']
    r_DHW_gen_PU = spec['PU_exaust_rate']['HotWater']
    r_HWH_gen_PU = spec['PU_exaust_rate']['WaterHeating']
    
    if exaust_use_WH == 'No':
        _Q_gen_DHW = min(q_pu_gen * r_DHW_gen_PU, l_dhw)
        _Q_gen_HWH = 0.0
    elif exaust_use_WH == 'Yes' and exaust_priority == 'HotWater':
        _Q_gen_DHW = min(q_pu_gen * r_DHW_gen_PU, l_dhw)
        _Q_gen_HWH = min((q_pu_gen - _Q_gen_DHW) * r_HWH_gen_PU, l_hwh_d)
    elif exaust_use_WH == 'Yes' and exaust_priority == 'WaterHeating':
        _Q_gen_HWH = min(q_pu_gen * r_HWH_gen_PU, l_hwh_d)
        _Q_gen_DHW = min((q_pu_gen - _Q_gen_HWH) * r_DHW_gen_PU, l_dhw)
    else:
        raise ValueError('exaust_use_WH is wrong.')
        
    return _Q_gen_DHW, _Q_gen_HWH

### 10.2 Amount of Generation / 発電量

The hourly power generation $E_{E,gen,PU,t}$ of power generation unit at $t$ o'clock is calculated by the equation below.

$$\large
E_{E,gen,PU,t} = E_{E,gen,PU} \times \frac{E_{E,PU,t}}{E_{E,PU}}
\tag{13}
$$

Where  
$E_{E,gen,PU}$ is the daily power generation of power generation unit. / 1日当たりの発電ユニットの発電量(kWh/day)  
$E_{E,PU,t}$ is the hourly sharing power load of power generation unit. / 1時間当たりの発電ユニットの分担可能電力負荷(kWh/h)  
$E_{E,PU}$ is the daily power load that can be shared by power generation unit. / 1日当たりの発電ユニットの分担可能電力負荷(kWh/day)  

The power generation $E_{E,gen,PU}$ of power generation unit is calculated by the equation below.

$$\large
E_{E,gen,PU} = E_{F,PU} \times e_{E,PU} \div 3.6
\tag{14}
$$

Where  
$E_{F,PU}$ is the daily fuel consumption of power generation unit. / 1日当たりの発電ユニットの燃料消費量(MJ/day)  
$e_{E,PU}$ is the daily average power generation efficiency of power generation unit. / 発電ユニットの日平均発電効率  

In [13]:
# 式(13)
def get_E_E_gen_PU(_E_F_PU, _e_E_PU, share_pload):
    _E_E_gen_PU = _E_F_PU * _e_E_PU / 3.6
    share_pload_daily = get_e_e_pu_d(share_pload)
    return [_E_E_gen_PU * share_pload[t] / share_pload_daily for t in range(24)]

### 10.3 Fuel Consumption / 燃料消費量

The daily fuel consumption $E_{F,PU}$ of power generation unit is calculated by the equations below.  
The power generation method of the power generation unit is accordance with Appendix A.

&#9312; When the power generation method of the power generation unit is "heat main":  

$$\large
E_{F,PU} = min(E_{F,PU,EVt}, E_{F,PU,HVt})
\tag{15a}
$$

&#9313; When the power generation method of the power generation unit is "electricity main":  

$$\large
E_{F,PU} = E_{F,PU,EVt}
\tag{15b}
$$

Where  
$E_{F,PU,EVt}$ is the virtual daily fuel consumption at estimation of power generation amount by power generation unit. / 1日当たりの発電ユニットの発電量推定時の仮想燃料消費量(MJ/day)  
$E_{F,PU,HVt}$ is the virtual daily fuel consumption at estimation of exhaust heat quantity by power generation unit. / 1日当たりの発電ユニットの排熱量推定時の仮想燃料消費量(MJ/day)  

The virtual daily fuel consumption $E_{F,PU,EVt}$ at estimation of power generation amount by power generation unit is calculated by the equation below.

$$\large
E_{F,PU,EVt} = E_{E,gen,PU,EVt} \times 3.6 \div e_{E,PU} 
\tag{16}
$$

Where  
$E_{E,gen,PU,EVt}$ is the virtual daily generation amount at estimation of power generation amount by power generation unit. / 1日当たりの発電ユニットの発電量推定時の仮想発電量(kWh/day)  
$e_{E,PU}$ is the daily average power generation efficiency of power generation unit. / 発電ユニットの日平均発電効率  

The virtual daily generation amount $E_{E,gen,PU,EVt}$ at estimation of the power generation amount of the power generation unit is calculated by the equation below.

$$
\displaystyle
E_{E,gen,PU,EVt} = \frac{ \min ( a_{PU} \times E_{E,PU} \times 3.6 + a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH}+ b, E_{E,PU} \times c \times 3.6) }{3.6}
$$

Where  
$E_{E,PU}$ is the daily power load that can be shared by power generation unit. / 1日当たりの発電ユニットの分担可能電力負荷(kWh/day)  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)  

The coefficients $a_{PU}$, $a_{DHW}$, $a_{HWH}$, $b$ and $c$ are accordance with Appendix A.

The virtual daily fuel consumption $E_{F,PU,HVt}$ at estimation of exhaust heat quantity by power generation unit is calculated by the equation below only for models using the equation (17a).

$$
\displaystyle
E_{F,PU,HVt} = (a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH}) \times r_{H,gen,PU,HVt} \div e_{H,PU}
\tag{18}
$$

Where  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)  
$r_{H,gen,PU,HVt}$ is the virtual exhaust heat quantity upper limit ratio at estimation of exhaust heat quantity by power generation unit. / 発電ユニットの排熱量推定時の仮想排熱量上限比  
$e_{H,PU}$ is the daily average exhaust heat efficiency of power generation unit. / 発電ユニットの日平均排熱効率  

The coefficients $a_{DHW}$ and $a_{HWH}$ are accordance with Appendix A.

$$
\displaystyle
r_{H,gen,PU,HVt} = a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH} + b
$$

Where  
$r_{H,gen,PU,HVt}$ is the virtual exhaust heat quantity upper limit ratio at estimation of exhaust heat quantity by power generation unit / 発電ユニットの排熱量推定時の仮想排熱量上限比;  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷, MJ/d;  
$L_{HWH}$ is the daily heat load of hot water heating / 1日当たりの温水暖房の熱負荷, MJ/d.

The coefficients $a_{DHW}$, $a_{HWH}$ and $b$ are accordance with Appendix A.

In [14]:
def get_fuel_consumption(spec, l_dhw_d, l_hwh, pwr_gen_eff, exhst_h_eff, share_pload):
    
    def get_r_H_gen_PU_HVt(spec, l_dhw_d, l_hwh_d):
        a_DHW = spec['PU_UpperCoef']['a_DHW']
        a_HWH = spec['PU_UpperCoef']['a_HWH']
        b     = spec['PU_UpperCoef']['b']
        return a_DHW * l_dhw_d + a_HWH * l_hwh_d + b

    def get_E_F_PU_HVt(spec, l_dhw_d, l_hwh_d, _r_H_gen_PU_HVt, exhst_h_eff):
        a_DHW = spec['PU_virtualFuel']['a_DHW']
        a_HWH = spec['PU_virtualFuel']['a_HWH']
        return (a_DHW * l_dhw_d + a_HWH * l_hwh_d) * _r_H_gen_PU_HVt / exhst_h_eff

    def get_E_E_gen_PU_EVt(spec, _E_E_PU, _L_DHW, _L_HWH):
        a_PU = spec['PU_virtualGeneration']['a_PU']
        a_DHW = spec['PU_virtualGeneration']['a_DHW']
        a_HWH = spec['PU_virtualGeneration']['a_HWH']
        b = spec['PU_virtualGeneration']['b']
        c = spec['PU_virtualGeneration']['c']
        p = a_PU * _E_E_PU * 3.6 + a_DHW * _L_DHW + a_HWH * _L_HWH + b
        q = _E_E_PU * c * 3.6
        return min(p, q) / 3.6

    # daily hot water heating demand, MJ/d (eq.30)
    l_hwh_d = get_L_HWH(l_hwh)
    
    # daily shareable power load, kWh/h
    share_pload_d = get_e_e_pu_d(share_pload)

    # virtual exhaust heat quantity upper limit ratio at estimation
    # of exhaust heat quantity by power generation unit (eq.19)
    _r_H_gen_PU_HVt = get_r_H_gen_PU_HVt(spec, l_dhw_d, l_hwh_d)

    # virtual daily fuel consumption at estimation
    # of exhaust heat quantity by power generation unit, MJ/d  
    _E_F_PU_HVt = get_E_F_PU_HVt(spec, l_dhw_d, l_hwh_d, _r_H_gen_PU_HVt, exhst_h_eff)

    _E_E_gen_PU_EVt = get_E_E_gen_PU_EVt(spec, share_pload_d, l_dhw_d, l_hwh_d)

    _E_F_PU_EVt = _E_E_gen_PU_EVt * 3.6 / pwr_gen_eff
    
    generate_use = spec['PU_GenerationMethod']
    
    if generate_use == 'HeatPriority':
        return min(_E_F_PU_EVt, _E_F_PU_HVt)
    elif generate_use == 'ElectricityPriority':
        return _E_F_PU_EVt
    else:
        raise ValueError('_generate_use に不正な値が指定されました。')

### 4. Power Generation Efficiency / 発電効率

$$
\displaystyle
e_{E,PU} = a_{PU} \times E_{E,PU} \times 3.6 + a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH} + b
$$

The value obtained by this equation is assumed to be equal to the upper limit value when it exceeds the upper limit value of the power generation efficiency of the power generation unit and equal to the lower limit value if it is lower than the lower limit value.

Where 
$e_{E,PU}$ is the daily average generation efficiency of power generation unit / 発電ユニットの日平均発電効率;  
$E_{E,PU}$ is the daily power load that can be shared by power generation unit / 1日当たりの発電ユニットの分担可能電力負荷, kWh/d  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷, MJ/d  
$L_{HWH}$ is the daily heat load of hot water heating / 1日当たりの温水暖房の熱負荷, MJ/d.  

The coefficients $a_{PU}$, $a_{DHW}$, $a_{HWH}$, $b$, the upper limit value and the lower limit value are accordance with Appendix A.

In [15]:
# 式(20)
def get_e_E_PU(spec, _E_E_PU, _L_DHW, _L_HWH):
    a_PU = spec['PU_GenIta']['a_PU']
    a_DHW = spec['PU_GenIta']['a_DHW']
    a_HWH = spec['PU_GenIta']['a_HWH']
    b = spec['PU_GenIta']['b']
    upperValue = spec['PU_GenIta']['UpperLimit']
    lowerValue = spec['PU_GenIta']['LowerLimit']
    
    e_E_PU = a_PU * _E_E_PU * 3.6 + a_DHW * _L_DHW + a_HWH * _L_HWH + b
    
    return max(min(e_E_PU, upperValue), lowerValue)
    

#### Example

In [16]:
get_e_E_PU(get_default_spec('PEFC5'), _E_E_PU = 10, _L_DHW = 30, _L_HWH = 50)

0.259756

### 10.5 Exhaust Heat Efficiency / 排熱効率

$$
\displaystyle
e_{H,PU} = a_{PU} \times E_{E,PU} \times 3.6 + a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH} + b
$$

The value obtained by this equation is assumed to be equal to the upper limit value when it exceeds the upper limit value of the exhaust heat efficiency of the power generation unit and equal to the lower limit value if it is lower than the lower limit value.

Where  
$e_{H,PU}$ is the daily average exhaust heat efficiency of power generation unit / 発電ユニットの日平均排熱効率;  
$E_{E,PU}$ is the daily power load that can be shared by power generation unit / 1日当たりの発電ユニットの分担可能電力負荷, kWh/d;  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷, MJ/day;  
$L_{HWH}$ is the daily heat load of hot water heating / 1日当たりの温水暖房の熱負荷, MJ/d.  

The coefficients $a_{PU}$, $a_{DHW}$, $a_{HWH}$, $b$, the upper limit value and the lower limit value are accordance with Appendix A.

In [17]:
# 式(21)
def get_e_H_PU(spec, _E_E_PU, _L_DHW, _L_HWH):
    a_PU = spec['PU_ExaustIta']['a_PU']
    a_DHW = spec['PU_ExaustIta']['a_DHW']
    a_HWH = spec['PU_ExaustIta']['a_HWH']
    b = spec['PU_ExaustIta']['b']
    upperValue = spec['PU_ExaustIta']['UpperLimit']
    lowerValue = spec['PU_ExaustIta']['LowerLimit']
    
    e_H_PU = a_PU * _E_E_PU * 3.6 + a_DHW * _L_DHW + a_HWH * _L_HWH + b
    
    return max(min(e_H_PU, upperValue), lowerValue)
    

In [18]:
get_e_H_PU(spec = get_default_spec('PEFC5'), _E_E_PU = 20, _L_DHW = 3, _L_HWH = 4)

0.30305600000000005

### 10.6 Shareable Electric Load / 分担可能電力負荷

The daily power load $E_{E,PU}$ that can be shared by power generation unit is calculated by the equation below.

$$
\displaystyle
E_{E,PU} = \sum_t E_{E,PU,t}
$$

Where  
$E_{E,PU,t}$ is the hourly power load that can be shared by power generation unit at $t$ o'clock / 時刻$t$における1時間当たりの発電ユニットの分担可能電力負荷, kWh/h.  

In [19]:
# 式(22)
def get_e_e_pu_d(e_e_pu):
    return sum(e_e_pu)

The hourly power load $E_{E,PU,t}$ that can be shared by power generation unit at $t$ o'clock is calculated by the equation below.

$$
\displaystyle
E_{E,PU,t} = \min \left( E_{E,dmd,PU,t}, P_{rtd,PU} \times 10^{-3} \right)
$$

The rated power generation output $P_{rtd,PU}$ is accordance with Appendix A.

$$
\displaystyle
E_{E,dmd,PU,t} = E_{E,dmd,t} + E_{E,TU,aux,t}
$$

Where  
$E_{E,PU,t}$ is the hourly power load that can be shared by power generation unit at $t$ o'clock / 時刻$t$における1時間当たりの発電ユニットの分担可能電力負荷, kWh/h;  
$E_{E,dmd,PU,t}$ is the hourly power demand of power generation unit at $t$ o'clock / 時刻$t$における1時間当たりの発電ユニットの電力需要, kWh/h;  
$P_{rtd,PU}$ is the rated power generation output / 定格発電出力, W;  
$E_{E,dmd,t}$ is the hourly power demand / 1時間当たりの電力需要, kWh/h;  
$E_{E,TU,aux,t}$ is the hourly auxiliary machine power consumption of the tank unit / 1時間当たりのタンクユニットの補機消費電力量, kWh/h.  

In [20]:
def get_shareable_pl(spec, power_demand, tu_aux_pc):
    
    P_rtd_PU = spec['RatedPower']

    # demand, kWh/h (eq.24)
    e_e_dmd_pu = [power_demand[t] + tu_aux_pc[t] for t in range(24)]
    
    # hourly shareable power load, kWh/h (eq.23)
    return [min(e_e_dmd_pu[t], P_rtd_PU * 10**(-3)) for t in range(24)]

## 12.Power Consumption of Auxiliaries of Tank Unit / タンクユニットの補機消費電力

The hourly power consumption $E_{E,TU,aux}$ of auxiliaries of tank unit is calculated by the equation below.

$$
\displaystyle
E_{E,TU,aux,t} = E_{E,TU,aux,DHW,t} + E_{E,TU,aux,HWH,t} + E_{E,TU,aux,ba2,t}
$$

In [21]:
# 式(25)
def get_e_e_tu_aux(e_e_tu_aux_dhw, e_e_tu_aux_hwh, e_e_tu_aux_ba2):
    return [e_e_tu_aux_dhw[t] + e_e_tu_aux_hwh[t] + e_e_tu_aux_ba2[t] for t in range(24)]

$$
\displaystyle
E_{E,TU,aux,DHW,t} = P_{TU,aux,DHW} \times 10^{-3}
$$

In [22]:
# 式(26)
def get_e_e_tu_aux_dhw(spec):
    return [spec['TU_auxPower']['HotWater'] * 10**(-3) for t in range(24)]

①温水暖房への排熱利用がある場合
$$
\displaystyle
E_{E,TU,aux,HWH,t} = P_{TU,aux,HWH} \times r_{WS,HWH,t} \times 10^{-3}
$$

In [23]:
# 式(27)
def get_e_e_tu_aux_hwh(spec, l_hwh, r_ws_hwh, temp_sw):

    exaust_use_WH = spec['ExaustUseForWaterHeating']
    P_TU_aux_HWH = spec['TU_auxPower']['WaterHeating']
    _equipment = spec['BB_WH']['Type']
    _e_rtd = spec['BB_WH']['RatedEfficiency']
    _q_rtd_hs = spec['BB_WH']['RatedPower']
        
    if exaust_use_WH == 'Yes':
        return [P_TU_aux_HWH * r_WS_HWH_t * 10**(-3) for r_WS_HWH_t in r_ws_hwh]
    elif exaust_use_WH == 'No':
        q_gen_hwh = 0.0 #温水暖房への排熱利用がないため0となる。

        hs_type = spec['BB_WH']['Type']
        _Ita = spec['BB_WH']['RatedEfficiency']
        _P_rtd = spec['BB_WH']['RatedPower']
        
        q_out_bb_hwh = get_q_out_bb_hwh(spec, l_hwh, q_gen_hwh)
        temp_rw = 40 # 暫定
        if hs_type == 'kerosene': # 付録A　石油熱源機
            _E_G_hs = [_04_07_AppA_ker.get_kerosene_cons(hs_type, q, t, temp_rw, _Ita, _P_rtd) for (q, t) in zip(q_out_bb_hwh, temp_sw)]
        elif hs_type == 'gasCondensing' or hs_type == 'gasConv': # 付録B ガス熱源機
            _E_G_hs = [_04_07_AppB_gas.get_gas_cons(hs_type, q, t, temp_rw, _Ita, _P_rtd) for (q, t) in zip(q_out_bb_hwh, temp_sw)]

        return [_04_07_AppB_gas.get_elec_cons(_equipment, r_WS_HWH_t, _E_G_hs_t) for (r_WS_HWH_t, _E_G_hs_t) in zip(r_ws_hwh, _E_G_hs)]
    else:
        raise ValueError('不正な値が入力されています。')


$$
\displaystyle
E_{E,TU,aux,ba2,t} = \left\{ \begin{array}{ll}
    E_{E,BB,aux,ba2} & (t = 19)\\ 
    0 & (t \neq 0)\\ 
    \end{array}
\right.
$$

In [24]:
# 式(28)
def get_e_e_tu_aux_ba2(spec, l2d_ba2):
    
    def get_e_e_tu_aux_ba2_t(_BB_WH_Type, t, l2d_ba2):
        if t == 19:
            if _BB_WH_Type == 'gasCondensing' or _BB_WH_Type == 'gasConv':
                return _07_AppC_gas.get_E_E_hs_aux3(l2d_ba2)
            elif _BB_WH_Type == 'kerCondensing' or _BB_WH_Type == 'gasConv':
                return _07_AppD_ker.get_E_E_hs_aux3(l2d_ba2)
            else:
                raise ValueError('Error: Back up boiler type is wrong')
        else:
            return 0.0
        
    _BB_WH_Type = spec['BB_WH']['Type']
    
    return [get_e_e_tu_aux_ba2_t(_BB_WH_Type, t, l2d_ba2) for t in range(24)]    

#### Exapmle

In [25]:
get_e_e_tu_aux_ba2(get_default_spec('SOFC1'), l2d_ba2=10.0)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.06480277777777778,
 0.0,
 0.0,
 0.0,
 0.0]

## Others / その他

### 1) Daily Hot Water Supply Heat Load corrected for solar heat except rewarming Bathtub / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷

$$ \displaystyle
L_{DHW} = L''_{k} + L''_{w} + L''_{s} + L''_{b1} + L''_{b2} + L''_{ba1}
$$

Where  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷, MJ/d;  
$ L''_{k} $ is the daily hot water supply heat load corrected for solar heat by the kitchen / 1日当たりの台所水栓の太陽熱補正給湯熱負荷, MJ/d;  
$ L''_{w} $ is the daily hot water supply heat load corrected for solar heat by the washroom / 1日当たりの洗面水栓の太陽熱補正給湯熱負荷, MJ/d;  
$ L''_{s} $ is the daily hot water supply heat load corrected for solar heat by the shower / 1日当たりの浴室シャワー水栓の太陽熱補正給湯熱負荷, MJ/d;  
$ L''_{b1} $ is the daily hot water supply heat load corrected for solar heat by filling the bathtub manually / 1日当たりの浴槽水栓湯はりの太陽熱補正給湯熱負荷, MJ/d;  
$ L''_{b2} $ is the daily hot water supply heat load corrected for solar heat by filling the bathtub automattically / 1日当たりの浴槽自動湯はりの太陽熱補正給湯熱負荷, MJ/d;  
$ L''_{ba1} $ is the daily hot water supply heat load corrected for solar heat by addition of hot water to bathtub / 1日当たりの浴槽水栓さし湯の太陽熱補正給湯熱負荷, MJ/d.

$L''_k$, $L''_s$, $L''_w$, $L''_{b1}$, $L''_{b2}$, $L''_{ba1}$ and $L''_{ba2}$ are calculated by Chapter 7.

In [26]:
# 式(29)
def get_l_dhw_d(l2d_k, l2d_w, l2d_s, l2d_b1, l2d_b2, l2d_ba1):
    return l2d_k + l2d_w + l2d_s + l2d_b1 + l2d_b2 + l2d_ba1

### 2) Daily Heat Load of Hot Water Heating / 1日当たりの温水暖房の熱負荷

$$
\displaystyle
L_{HWH} = \sum^{23}_{t=0}L_{HWH,t}
$$

Where  
$L_{HWH}$ is the daily heat load of hot water heating / 1日当たりの温水暖房の熱負荷, MJ/d;  
$L_{HWH,t}$ is the heat load of hot water heating per hour at $t$ o'clock. / ݇時刻$t$における1 時間当たりの温水暖房の熱負荷, MJ/d.

In [27]:
# 式(30)
def get_L_HWH(L_HWH_t):
    return np.sum(L_HWH_t)

### 3) Hourly Power Demand / 1時間当たりの電力需要

$$
\displaystyle
E_{E,dmd} = E_{E,H} + E_{E,C} + E_{E,V} + E_{E,L} + E_{E,AP}
$$

Where  
$E_{E,dmd}$ is the hourly power demand / 1時間当たりの電力需要, kWh/h;  
$E_{E,H}$ is the hourly power consumption of heating equipment / 1時間当たりの暖房設備の消費電力量, kWh/h;  
$E_{E,C}$ is the hourly power consumption of cooling equipment / 1時間当たりの冷房設備の消費電力量, kWh/h;  
$E_{E,V}$ is the hourly power consumption of mechanical ventilation equipment / 1時間当たりの機械換気設備の消費電力量, kWh/h;  
$E_{E,L}$ is the hourly power consumption of lighting equipment / 1時間当たりの照明設備の消費電力量, kWh/h;  
$E_{E,AP}$ is the hourly power consumption of home appliances / 1時間当たりの家電の消費電力量, kWh/h.  

In [28]:
# 式(31)
def get_power_demand(e_e_h, e_e_c, e_e_v, e_e_l, e_e_ap):
    return [h+c+v+l+ap for (h,c,v,l,ap) in zip(e_e_h, e_e_c, e_e_v, e_e_l, e_e_ap)]

## Integrated Function

### 1) Power Consumption of Auxiliaries of Tank Unit / タンクユニットの補機消費電力

In [29]:
# hourly tank unit's auxiliaries power consumption, kWh/h
def get_tu_aux_pc(spec, l2d_ba2, l_hwh, r_ws_hwh, theta_sw_hwh):
    
    # for rewarming bathtub
    e_e_tu_aux_ba2 = get_e_e_tu_aux_ba2(spec, l2d_ba2)
    
    # for hot water supply
    e_e_tu_aux_dhw = get_e_e_tu_aux_dhw(spec)
    
    # for hot water heating
    e_e_tu_aux_hwh = get_e_e_tu_aux_hwh(spec, l_hwh, r_ws_hwh, theta_sw_hwh)
    
    # total
    return get_e_e_tu_aux(e_e_tu_aux_dhw, e_e_tu_aux_hwh, e_e_tu_aux_ba2)

### 3) 発電ユニットの発電効率・排熱効率

In [30]:
# power generation and exhaust heat efficiency of power unit
def get_pu_efficienty(spec, share_pload, l_dhw_d, l_hwh):
    
    # daily shareable electric load, kWh/h
    share_pload_daily = get_e_e_pu_d(share_pload)

    # daily hot water heating demand
    l_hwh_d = get_L_HWH(l_hwh)
    
    # power generation efficiency
    pwr_gen_eff= get_e_E_PU(spec, share_pload_daily, l_dhw_d, l_hwh_d)

    # exhaust heat efficiency
    exhst_h_eff = get_e_H_PU(spec, share_pload_daily, l_dhw_d, l_hwh_d)    
    
    return pwr_gen_eff, exhst_h_eff

In [31]:
def get_result(cgs_name, theta_ex,
               e_e_h, e_e_c, e_e_v, e_e_l, e_e_ap,
               l_hwh,
               l2d_k, l2d_w, l2d_s, l2d_b1, l2d_b2, l2d_ba1, l2d_ba2,
               r_ws_hwh, theta_sw_hwh,
               is_hwh_connected
              ):
    
    spec = get_default_spec(cgs_name)
    
    # hourly power demand, kWh/h
    power_demand = get_power_demand(e_e_h, e_e_c, e_e_v, e_e_l, e_e_ap)
    
    # daily hot water supply demand except rewarming bathtub
    l_dhw_d = get_l_dhw_d(l2d_k, l2d_w, l2d_s, l2d_b1, l2d_b2, l2d_ba1)
    
    # データの事前処理：None -> 0
    if not is_hwh_connected:
        l_hwh = [0.0 for t in range(24)]
        r_ws_hwh = [0.0 for t in range(24)]
        theta_sw_hwh = [0.0 for t in range(24)]
        
    # hourly tank unit's auxiliaries power consumption, kWh/h
    tu_aux_pc = get_tu_aux_pc(spec, l2d_ba2, l_hwh, r_ws_hwh, theta_sw_hwh)
    
    # hourly shareable power load, kWh/h
    share_pload = get_shareable_pl(spec, power_demand, tu_aux_pc)
    
    # power generation and exhaust heat efficiency of power unit
    pwr_gen_eff, exhst_h_eff = get_pu_efficienty(spec, share_pload, l_dhw_d, l_hwh)
    
    # daily power unit fuel consumption, MJ/d
    fuel_cons_pu = get_fuel_consumption(spec, l_dhw_d, l_hwh, pwr_gen_eff, exhst_h_eff, share_pload)
    
    # hourly power unit power generation, kWh/h
    pow_gen_pu = get_E_E_gen_PU(fuel_cons_pu, pwr_gen_eff, share_pload)
    
    # 排熱利用量
    _Q_gen_DHW, _Q_gen_HWH = get_Q_gen_DHWandHWH(fuel_cons_pu, exhst_h_eff, spec, l_dhw_d, l_hwh)
    
    # 温水暖房時のBBの燃料消費量
    _E_F_BB_HWH = get_bb_hwh_fuel_cons(l_hwh, _Q_gen_HWH, spec, theta_sw_hwh)
    
    # 日平均外気温度
    temp_ex_ave = sum(theta_ex) / len(theta_ex)
    
    # fuel consumption of backup boiler, MJ/d
    _E_F_BB_DHW = get_bb_fuel_cons(spec, temp_ex_ave, l2d_k, l2d_s, l2d_w, l2d_b1, l2d_b2, l2d_ba1, l2d_ba2, l_dhw_d, _Q_gen_DHW)
    
    # 給湯時のBBの年間平均効率
    # _e_BB_ave = get_e_BB_ave(_L_BB_DHW, _E_F_BB_DHW)
    
    # power generation, kWh/h
    pow_gen = get_pow_gen(pow_gen_pu, tu_aux_pc)
    
    # fuel consumption, MJ/h
    fuel_cons = get_energy_cons(fuel_cons_pu, _E_F_BB_DHW, _E_F_BB_HWH )
    
    return pow_gen, fuel_cons    

In [32]:
# 1/1のデータ(テストケース：CGA15：主居室に温水暖房設置、PEFC5)

cgs_name = 'PEFC5'

theta_ex = [0.1,-0.4,-0.4,-0.9,-1.2,-1.3,-1.7,-1.4,-1.5,-1.0,-0.7,-0.1,0.6,0.0,-0.6,-0.8,-1.2,-1.0,-1.1,-0.9,-0.7,-0.4,-0.4,-0.3]

e_e_h  = [0.000000000, 0.000000000, 0.000000000, 0.000000000, 0.000000000, 0.000000000, 0.000000000, 0.935374721, 1.172647829, 0.550253109, 0.461841104, 0.191380807, 0.000000000, 0.000000000, 0.000000000, 0.396192608, 0.272354377, 0.279698030, 0.000000000, 0.965689165, 0.531700445, 0.480495066, 0.000000000, 0.000000000]
e_e_c  = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
e_e_v  = [0.04805168, 0.04805168, 0.04805168, 0.04805168, 0.04805168, 0.04805168, 0.04805168, 0.04972168, 0.06273168, 0.04888168, 0.04972168, 0.04888168, 0.06106168, 0.04888168, 0.04805168, 0.04805168, 0.04972168, 0.07323168, 0.07323168, 0.04805168, 0.04888168, 0.07267168, 0.07350168, 0.06078168]
e_e_l  = [0.00000000, 0.00000000, 0.00000000, 0.00000000, 0.00000000, 0.00000000, 0.00000000, 0.04733837, 0.22800415, 0.44740201, 0.37772274, 0.34025132, 0.30012517, 0.18795654, 0.00000000, 0.00000000, 0.14064281, 0.26009139, 0.31921666, 0.13532806, 0.28202169, 0.32578565, 0.32982943, 0.26238486]
e_e_ap = [0.1812, 0.0717, 0.0795, 0.0795, 0.0717, 0.0795, 0.0795, 0.2774, 0.3702, 0.3272, 0.3134, 0.3157, 0.3705, 0.0717, 0.0795, 0.0795, 0.5453, 0.5187, 0.4476, 0.2056, 0.219, 0.3465, 0.3009, 0.1812]

l_hwh  = [6.970726745, 7.099828411, 7.143584395, 7.265929805, 7.371793644, 7.441854544, 7.529509219, 6.921503462, 4.765329118, 4.209378584, 3.442025497, 1.491250810, 1.305083978, 3.904003285, 4.996332889, 4.823518251, 8.515275931, 8.915011810, 5.149533946, 5.482711577, 5.529148880, 5.861759040, 7.106735192, 7.216744703]

l2d_k = 27.71142793
l2d_w = 6.92785698
l2d_s = 36.14534078
l2d_b1 = 0.0
l2d_b2 = 27.10900559
l2d_ba1 = 0.0
l2d_ba2 = 7.60812500

r_ws_hwh= [0.520002758, 0.529412701, 0.532675453, 0.541572611, 0.549325924, 0.55450047 , 0.560844811, 0.515685597, 0.35501057 , 0.313723672, 0.256597235, 0.111225877, 0.097397365, 0.291206818, 0.372499297, 0.359555235, 0.6345351  , 0.664433047, 0.383761177, 0.408658837, 0.412188787, 0.437093618, 0.529927719, 0.538175666]

theta_sw_hwh = [60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60]

_E_E_CG_gen_tArr, _E_CG = get_result(cgs_name = cgs_name,
                                     theta_ex = theta_ex,
                                     e_e_h = e_e_h,
                                     e_e_c = e_e_c,
                                     e_e_v = e_e_v,
                                     e_e_l = e_e_l,
                                     e_e_ap = e_e_ap,
                                     l_hwh = l_hwh,
                                     l2d_k = l2d_k,
                                     l2d_w = l2d_w,
                                     l2d_s = l2d_s,
                                     l2d_b1 = l2d_b1,
                                     l2d_b2 = l2d_b2,
                                     l2d_ba1 = l2d_ba1,
                                     l2d_ba2 = l2d_ba2,
                                     r_ws_hwh = r_ws_hwh,
                                     theta_sw_hwh = theta_sw_hwh,
                                     is_hwh_connected = True
                                    )

# 期待値
Exp_E_E_CG_gen_tArr = (0.202042, 0.101845, 0.108936, 0.108830, 0.101609, 0.108677, 0.108601, 0.553198, 0.575387, 0.581089, 0.588978, 0.609054, 0.610964, 0.277228, 0.110838, 0.473120, 0.536785, 0.532656, 0.571417, 0.514624, 0.567491, 0.564052, 0.551231, 0.453287)
Exp_E_CG = 419.215252

print("小数点以下6桁によるチェック：E_E_CG_gen_tArr:", np.round(_E_E_CG_gen_tArr,6) - Exp_E_E_CG_gen_tArr, "E_CG:", np.round(_E_CG,6) - Exp_E_CG)


小数点以下6桁によるチェック：E_E_CG_gen_tArr: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] E_CG: 0.0


In [33]:
# 1/1のデータ(テストケース：CGA28：主居室に床暖房設置、SOFC2)

cgs_name = 'SOFC2'

theta_ex = [2.6,2.9,2.8,2.4,2.4,2.8,2.3,2.3,3.7,5.9,7.7,8.5,9.3,9.8,9.6,8.9,7.9,6.9,6.4,6.4,6.4,6.2,6.1,5.9]

e_e_h  = [0,0,0,0,0,0,0,0.633023336,0.57736388,0,0,0,0,0,0,0,0,0.121440137,0,0.34838431,0.289969246,0.285920062,0,0]
e_e_c  = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
e_e_v  = [0.028013333,0.028013333,0.028013333,0.028013333,0.028013333,0.028013333,0.028013333,0.028983333,0.036573333,0.0285,0.028983333,0.0285,0.035596667,0.0285,0.028013333,0.028013333,0.028983333,0.0427,0.0427,0.028013333,0.0285,0.042373333,0.042863333,0.035433333]
e_e_l  = [0,0,0,0,0,0,0,0.06520215,0.161473339,0.216220292,0.139673577,0.134740187,0.149334927,0.100393167,0,0,0.018619536,0.06315692,0.166700676,0.14517338,0.14625549,0.133016709,0.149577155,0.109285714]
e_e_ap = [0.1654,0.0559,0.063666667,0.063666667,0.0559,0.063666667,0.063666667,0.215833333,0.310666667,0.276833333,0.255866667,0.2618,0.305066667,0.0559,0.063666667,0.063666667,0.3152,0.308366667,0.2444,0.158633333,0.157766667,0.276,0.219733333,0.1654]

l_hwh  = [0,0,0,0,0,0,0,12.41471107,6.652157304,0.215525797,0,0,0,0,0,0,2.794573005,6.345026647,5.54776174,6.425939599,6.848676137,7.49334577,0,0]

l2d_k = 20.760032
l2d_w = 5.437151
l2d_s = 23.231464
l2d_b1 = 0.0
l2d_b2 = 26.691470
l2d_ba1 = 0.0
l2d_ba2 = 4.042674

r_ws_hwh = [0,0,0,0,0,0,0,1,0.536344948,0.034472133,0,0,0,0,0,0,0.448200141,0.512711154,0.887939355,0.519070306,0.553217839,0.605209067,0,0]

theta_sw_hwh = [40,40,40,40,40,40,40,60,60,40,40,40,40,40,40,40,40,60,40,60,60,60,40,40]

_E_E_CG_gen_tArr, _E_CG = get_result(cgs_name = cgs_name,
                                     theta_ex = theta_ex,
                                     e_e_h = e_e_h,
                                     e_e_c = e_e_c,
                                     e_e_v = e_e_v,
                                     e_e_l = e_e_l,
                                     e_e_ap = e_e_ap,
                                     l_hwh = l_hwh,
                                     l2d_k = l2d_k,
                                     l2d_w = l2d_w,
                                     l2d_s = l2d_s,
                                     l2d_b1 = l2d_b1,
                                     l2d_b2 = l2d_b2,
                                     l2d_ba1 = l2d_ba1,
                                     l2d_ba2 = l2d_ba2,
                                     r_ws_hwh = r_ws_hwh,
                                     theta_sw_hwh = theta_sw_hwh,
                                     is_hwh_connected = True
                                    )

# 期待値
Exp_E_E_CG_gen_tArr = (0.177268, 0.076383, 0.083539, 0.083539, 0.076383, 0.083539, 0.083539, 0.548079, 0.587175, 0.479347, 0.390196, 0.390672, 0.450519, 0.169326, 0.083539, 0.083539, 0.330529, 0.489134, 0.411648, 0.552352, 0.568862, 0.581382, 0.378817, 0.284792)
Exp_E_CG = 227.176121

print("小数点以下6桁によるチェック：E_E_CG_gen_tArr:", np.round(_E_E_CG_gen_tArr,6) - Exp_E_E_CG_gen_tArr, "E_CG:", np.round(_E_CG,6) - Exp_E_CG)


小数点以下6桁によるチェック：E_E_CG_gen_tArr: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] E_CG: 0.0


In [34]:
# 1/1のデータ(テストケース：CGA44：温水暖房なし、SOFC1)

cgs_name = 'SOFC1'

theta_ex = [0.1,-0.4,-0.4,-0.9,-1.2,-1.3,-1.7,-1.4,-1.5,-1.0,-0.7,-0.1,0.6,0.0,-0.6,-0.8,-1.2,-1.0,-1.1,-0.9,-0.7,-0.4,-0.4,-0.3]

e_e_h  = [0.000000000, 0.000000000, 0.000000000, 0.000000000, 0.000000000, 0.000000000, 0.000000000, 4.421937853, 2.056425869, 1.228938622, 1.013728171, 0.560950824, 0.324964088, 0.000000000, 0.000000000, 1.425335220, 1.183681359, 1.182490965, 0.605775629, 1.581011581, 1.137740362, 1.098609600, 0.000000000, 0.000000000]
e_e_c  = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
e_e_v  = [0.04805168, 0.04805168, 0.04805168, 0.04805168, 0.04805168, 0.04805168, 0.04805168, 0.04972168, 0.06273168, 0.04888168, 0.04972168, 0.04888168, 0.06106168, 0.04888168, 0.04805168, 0.04805168, 0.04972168, 0.07323168, 0.07323168, 0.04805168, 0.04888168, 0.07267168, 0.07350168, 0.06078168]
e_e_l  = [0.00000000, 0.00000000, 0.00000000, 0.00000000, 0.00000000, 0.00000000, 0.00000000, 0.04733837, 0.22800415, 0.44740201, 0.37772274, 0.34025132, 0.30012517, 0.18795654, 0.00000000, 0.00000000, 0.14064281, 0.26009139, 0.31921666, 0.13532806, 0.28202169, 0.32578565, 0.32982943, 0.26238486]
e_e_ap = [0.1812, 0.0717, 0.0795, 0.0795, 0.0717, 0.0795, 0.0795, 0.2774, 0.3702, 0.3272, 0.3134, 0.3157, 0.3705, 0.0717, 0.0795, 0.0795, 0.5453, 0.5187, 0.4476, 0.2056, 0.219, 0.3465, 0.3009, 0.1812]

l_hwh  = [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None]

l2d_k = 27.71142793
l2d_w = 6.92785698
l2d_s = 36.14534078
l2d_b1 = 0.0
l2d_b2 = 27.10900559
l2d_ba1 = 0.0
l2d_ba2 = 7.60812500

r_ws_hwh = [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None]

theta_sw_hwh = [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None]

_E_E_CG_gen_tArr, _E_CG = get_result(cgs_name = cgs_name,
                                     theta_ex = theta_ex,
                                     e_e_h = e_e_h,
                                     e_e_c = e_e_c,
                                     e_e_v = e_e_v,
                                     e_e_l = e_e_l,
                                     e_e_ap = e_e_ap,
                                     l_hwh = l_hwh,
                                     l2d_k = l2d_k,
                                     l2d_w = l2d_w,
                                     l2d_s = l2d_s,
                                     l2d_b1 = l2d_b1,
                                     l2d_b2 = l2d_b2,
                                     l2d_ba1 = l2d_ba1,
                                     l2d_ba2 = l2d_ba2,
                                     r_ws_hwh = r_ws_hwh,
                                     theta_sw_hwh = theta_sw_hwh,
                                     is_hwh_connected = False
                                    )

# 期待値
Exp_E_E_CG_gen_tArr = (0.219993, 0.114608, 0.122115, 0.122115, 0.114608, 0.122115, 0.122115, 0.656590, 0.656590, 0.656590, 0.656590, 0.656590, 0.656590, 0.296299, 0.122115, 0.656590, 0.656590, 0.656590, 0.656590, 0.603235, 0.656590, 0.656590, 0.656590, 0.484767)
Exp_E_CG = 209.941962

print("小数点以下6桁によるチェック：E_E_CG_gen_tArr:", np.round(_E_E_CG_gen_tArr,6) - Exp_E_E_CG_gen_tArr, "E_CG:", np.round(_E_CG,6) - Exp_E_CG)



小数点以下6桁によるチェック：E_E_CG_gen_tArr: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] E_CG: 0.0
